# Join hand annotated Hypernym failure modes docs

### Imports & load files

In [ ]:
import pandas as pd

In [ ]:
df_scott = pd.read_csv('last_samples_to_code_stc.csv')
df_danny = pd.read_csv('last_samples_to_code - Danny (Complete).csv')
df_rich = pd.read_csv('last_samples_to_code - Rich (Complete).csv')
df_tv = pd.read_csv('df_tv_sample_v2.csv')
df_music = pd.read_csv('df_music_sample_v2.csv')
df_film = pd.read_csv('df_film_sample_v2.csv')
df_album = pd.read_csv('df_album_sample_v2.csv')

### Clean up inputs

In [ ]:
df_danny = df_danny[df_danny['assignee']=='danny']
df_danny.drop('Unnamed: 12', inplace=True, axis=1)
df_danny.drop('Unnamed: 13', inplace=True, axis=1)

In [ ]:
df_rich.drop('Unnamed: 0.1', inplace=True, axis=1)
df_rich = df_rich[df_rich['assignee']=='rich']

In [ ]:
df_complete = pd.concat([df_scott, df_danny, df_rich, df_tv, df_film, df_music, df_album], axis=0)

In [ ]:
df_complete.reset_index(drop=True, inplace=True)

In [ ]:
# down sample the human data points so that we have just 10% of errors included in the final results

df_human_resample = df_complete[df_complete['classLabel.value']=='human'].sample(n=399)

In [ ]:
human_index = df_complete[df_complete['classLabel.value']=='human'].index.values

In [ ]:
df_complete.drop(labels=human_index, axis=0, inplace=True)

In [ ]:
df_complete = pd.concat([df_complete, df_human_resample], axis=0)

### Build pivot tables for analysis

In [ ]:
cats_pivot = df_complete.pivot_table(values='classLabel.xml:lang', columns=['classLabel.value'], aggfunc='count')
cats_pivot = cats_pivot.transpose()

In [ ]:
complete_pivot = df_complete.pivot_table(values='classLabel.xml:lang', columns=['classLabel.value','error_category'], aggfunc='count')
complete_pivot = complete_pivot.transpose()

complete_pivot.reset_index(drop=False, inplace=True)

percents = list()
for cat in cats_pivot.index:
    col = complete_pivot[complete_pivot['classLabel.value'] == cat]['classLabel.xml:lang']
    percents.extend((col / col.sum()).values)

complete_pivot['percent'] = percents

### Review pivot

In [ ]:
complete_pivot

,classLabel.value,error_category,classLabel.xml:lang,percent
0,album,conflation,193,0.835498
1,album,denial,5,0.021645
2,album,phrase,7,0.030303
3,album,specification,26,0.112554
4,business,conflation,11,0.255814
5,business,specification,32,0.744186
6,film,conflation,120,0.779221
7,film,denial,3,0.019481
8,film,phrase,4,0.025974
9,film,specification,27,0.175325


### Save results

In [ ]:
df_complete.to_csv('last_samples_complete_all_v2.csv')

In [ ]:
complete_pivot.to_csv('complete_pivot_v2.csv')